In [1]:
from qdk_chemistry import Molecule

In [2]:
water = Molecule.from_smiles("O")

In [3]:
water.create_input(
    "water", 
    file_name="data/water.nw", 
    solver="nwchem", 
    num_tce_root=5, 
    num_active_orbitals=4
)

'./data/water.nw'

In [21]:
with open("data/water.output") as f:
    logfile = f.read()

with open("data/water.nw") as f:
    nwfile = f.read()

In [22]:
import re

In [26]:
!docker run -it -v "$(pwd)":"/opt/data" nwchemorg/nwchem-qc "data/water.nw"

 argument  1 = data/water.nw
  NWChem w/ OpenMP: maximum threads =    1



============================== echo of input deck ==============================

start water

echo
memory stack 1000 mb heap 100 mb global 1000 mb noverify

geometry units au
symmetry c1
O 0.008745998706025862 0.3977188702645636 0.0
H 0.758910362423615 -0.21564433665585545 0.0
H -0.7676563611296411 -0.18207453360870887 0.0
end

basis
* library sto-3g
end



scf
thresh 1.0e-08
tol2e 1e-09
singlet
rhf
maxiter 200

end

tce
ccsd
2eorb
2emet 13
tilesize 1
nroots 5
thresh 1.0e-08
end

set tce:print_integrals T
set tce:qorb 4
set tce:qela 2
set tce:qelb 2

task tce energy


                                         
                                         


              Northwest Computational Chemistry Package (NWChem) 6.8
              ------------------------------------------------------


                    Environmental Molecular Sciences Laboratory
                       Pacific Northwest National Laborator

     1   1 0 0     -0.017685      0.000000      0.000000
     1   0 1 0     -0.804202      0.000000      0.000000
     1   0 0 1      0.000000      0.000000      0.000000

     2   2 0 0     -3.463782      0.000000      1.165478
     2   1 1 0     -0.008383      0.000000     -0.013101
     2   1 0 1      0.000000      0.000000      0.000000
     2   0 2 0     -3.844794      0.000000      0.570013
     2   0 1 1      0.000000      0.000000      0.000000
     2   0 0 2     -4.165523      0.000000      0.000000


 Parallel integral file used       1 records with       0 large values

 ------------------------------------
     Number of active orbitals      4
     Number of active alpha electrons      2
     Number of active beta electrons       2
 ------------------------------------
                   NWChem Extensible Many-Electron Theory Module
                   ---------------------------------------------

                   This portion of the program was automatically
            

   0.0679273487627   1.0595218590524   28.83107
   0.0953587459826   1.0924316969279   29.72659
   0.0363270298857   1.4016504202612   38.14086
   0.0891744671688   1.5219796687304   41.41519     0.1     0.2

 Iteration   2 using    8 trial vectors
   0.0052607889635   1.0581976347605   28.79503
   0.0100821017819   1.0899513760008   29.65910
   0.0001256316769   1.4011453112373   38.12712
   0.0084154857156   1.5190205217788   41.33467     0.1     0.2

 Iteration   3 using   12 trial vectors
   0.0000000000000   1.0581886331441   28.79479
   0.0011347450981   1.0899124151704   29.65804
   0.0000000000000   1.4011449990503   38.12711
   0.0000000000156   1.5189944944448   41.33396     0.1     0.2

 Iteration   4 using   13 trial vectors
   0.0000000000000   1.0581886331441   28.79479
   0.0000026453332   1.0899123131413   29.65804
   0.0000000000000   1.4011449990503   38.12711
   0.0000000000094   1.5189944944448   41.33396     0.0     0.1

 Iteration   5 using   14 trial vectors
   0

In [11]:
import re
def parse_nwchem_output(nw_input_file_path, nw_output_file_path):
    with open(nw_output_file_path) as f:
        logfile = f.read()

    with open(nw_input_file_path) as f:
        nwfile = f.read()
    pattern_atom = "atoms           =\s*([0-9]*)"
    pattern_orbital = "Number of AO functions :\s*([0-9]*)"
    pattern_scf = "Total SCF energy =\s*([+-]?[0-9]*[.][0-9]+)"
    n_atom = int(re.findall(pattern_atom, logfile)[0])
    n_orbital = int(re.findall(pattern_orbital, logfile)[0])
    scf_energy = re.findall(pattern_scf, logfile)
    pattern_ccsd = "correlation energy / hartree =\s*([+-]?[0-9]*[.][0-9]+)"
    ccsd_corr_energy = re.findall(pattern_ccsd, logfile)

    if "optimize" in nwfile: # The following geometry snapshots doesn't include the initial one, but does include final one. 
        pattern_geom = "([0-9]+)\s*([A-Z]+)\s*([+-]?[0-9]*[.][0-9]+)\s*([+-]?[0-9]*[.][0-9]+)\s*([+-]?[0-9]*[.][0-9]+)\s*([+-]?[0-9]*[.][0-9]+)\s*([+-]?[0-9]*[.][0-9]+)\s*([+-]?[0-9]*[.][0-9]+)"
        geomsnapshot = re.findall(pattern_geom,logfile)
        for i in range(0,len(geomsnapshot)):
            if (i%n_atom==0):
                print("Geometry snapshot %d"%(i/n_atom))
            print(geomsnapshot[i][0:5])
    print("Number of Atoms: ", n_atom)
    print("Number of Orbitals: ", n_orbital)
    print("SCF Energy: ",scf_energy[-1])
    print("Correlation Energy: ",ccsd_corr_energy[-1])

In [5]:
caffeine = Molecule.from_xyz("data/caffeine.xyz")

In [21]:
caffeine.create_input(
    "caffeine", 
    "data/caffeine.nw", 
    solver="nwchem", 
    num_active_orbitals=80, 
    num_active_el=51*2,
    memory="memory stack 1000 mb heap 100 mb global 8000 mb noverify"
)

'./data/caffeine.nw'

In [22]:
cat data/caffeine.nw


start caffeine

echo
memory stack 1000 mb heap 100 mb global 8000 mb noverify

geometry units au
symmetry c1
H -2.56844271140696 2.6223282057988957 -0.6824886654803207
N -1.2713939613354759 -1.0944449903712032 0.013656799979650308
C 0.8252067608259046 0.05160171602104182 -0.0667250490775411
N -1.1591671902255938 1.2969194668121147 0.0851804771024143
C 0.23595237337498998 1.3426180260459684 0.008362881602143817
C -1.9448587032734757 0.12507818491670597 0.09114625146835935
C 0.10020160085450462 -1.1011504288407 -0.06354637437477963
N 2.126953215940903 -0.35111108224689025 -0.15289546056200093
O -3.1759709519690817 0.18746041114437595 0.16227998834009327
C -1.8805164911990553 2.5518698795245123 0.16651028313049213
O 0.8986762940669188 2.3757240014655188 0.004787350100643291
H 3.026864937437135 -2.310425513032107 -0.26716753213767763
C -2.0058222741700256 -2.3463670931767733 0.014138966599583308
H -2.46894964660055 2.564654699522161 1.0898677836023343
H -1.216962994139419 3.42003092793995

In [23]:
!docker run -it -v "$(pwd)/data":"/opt/data" nwchemorg/nwchem-qc "caffeine.nw"

 argument  1 = caffeine.nw
  NWChem w/ OpenMP: maximum threads =    1



============================== echo of input deck ==============================

start caffeine

echo
memory stack 1000 mb heap 100 mb global 8000 mb noverify

geometry units au
symmetry c1
H -2.56844271140696 2.6223282057988957 -0.6824886654803207
N -1.2713939613354759 -1.0944449903712032 0.013656799979650308
C 0.8252067608259046 0.05160171602104182 -0.0667250490775411
N -1.1591671902255938 1.2969194668121147 0.0851804771024143
C 0.23595237337498998 1.3426180260459684 0.008362881602143817
C -1.9448587032734757 0.12507818491670597 0.09114625146835935
C 0.10020160085450462 -1.1011504288407 -0.06354637437477963
N 2.126953215940903 -0.35111108224689025 -0.15289546056200093
O -3.1759709519690817 0.18746041114437595 0.16227998834009327
C -1.8805164911990553 2.5518698795245123 0.16651028313049213
O 0.8986762940669188 2.3757240014655188 0.004787350100643291
H 3.026864937437135 -2.310425513032107 -0.26716753213767763
C -

  autoz: too many torsions 
     4 autoz failed with cvr_scaling = 1.5 changing to 1.6
      warning. autoz generated    7 bonds for atom    1
      warning. autoz generated   15 bonds for atom    2
      warning. autoz generated   17 bonds for atom    3
      warning. autoz generated   14 bonds for atom    4
      warning. autoz generated   18 bonds for atom    5
      warning. autoz generated   18 bonds for atom    6
      warning. autoz generated   17 bonds for atom    7
      warning. autoz generated   14 bonds for atom    8
      warning. autoz generated   11 bonds for atom    9
      warning. autoz generated   11 bonds for atom   10
      warning. autoz generated   13 bonds for atom   11
      warning. autoz generated   12 bonds for atom   13
      warning. autoz generated    7 bonds for atom   14
      warning. autoz generated    7 bonds for atom   15
      warning. autoz generated    7 bonds for atom   16
      warning. autoz generated    7 bonds for atom   17
      warning. au

 H                           sto-3g                  1        1   1s
 N                           sto-3g                  3        5   2s1p
 C                           sto-3g                  3        5   2s1p
 O                           sto-3g                  3        5   2s1p


                                 NWChem SCF Module
                                 -----------------



  ao basis        = "ao basis"
  functions       =    80
  atoms           =    24
  closed shells   =    51
  open shells     =     0
  charge          =   0.00
  wavefunction    = RHF 
  input vectors   = atomic
  output vectors  = ./caffeine.movecs
  use symmetry    = F
  symmetry adapt  = F


 Summary of "ao basis" -> "ao basis" (cartesian)
 ------------------------------------------------------------------------------
       Tag                 Description            Shells   Functions and Types
 ---------------- ------------------------------  ------  ---------------------
 H                       

    13     -0.284083   4 N  s                25     -0.198356   6 C  py        

 Vector   46  Occ=2.000000D+00  E=-5.546402D-01
              MO Center= -1.3D+00,  2.2D-01,  7.8D-02, r^2= 2.0D+00
   Bfn.  Coefficient  Atom+Function         Bfn.  Coefficient  Atom+Function  
  ----- ------------  ---------------      ----- ------------  ---------------
    39      0.705255   9 O  px               23     -0.525725   6 C  s         
    38     -0.513057   9 O  s                55     -0.403286  13 C  px        
    24     -0.397389   6 C  px               44     -0.392794  10 C  px        
    29     -0.363142   7 C  px               50      0.309152  11 O  py        
    18      0.299658   5 C  s                19     -0.279223   5 C  px        

 Vector   47  Occ=2.000000D+00  E=-5.544621D-01
              MO Center= -7.0D-01,  7.5D-01,  6.4D-02, r^2= 1.6D+00
   Bfn.  Coefficient  Atom+Function         Bfn.  Coefficient  Atom+Function  
  ----- ------------  ---------------      ----- 


 Integral file          = ./caffeine.aoints.0
 Record size in doubles =  65536        No. of integs per rec  =  43688
 Max. records in memory =    166        Max. records in file   = 394975
 No. of bits per label  =      8        No. of bits per value  =     64


 #quartets = 8.882D+05 #integrals = 5.225D+06 #direct =  0.0% #cached =100.0%


 Fock matrix recomputed
 1-e file size   =             6400
 1-e file name   = ./caffeine.f1
 Cpu & wall time / sec            5.5            5.6
 4-electron integrals stored in orbital form

 v2    file size   =          5250420
 4-index algorithm nr.  13 is used
 imaxsize =       30
 imaxsize ichop =        0

Program received signal SIGBUS: Access to an undefined portion of a memory object.

Backtrace for this error:
#0  0x7fd8d382e2da in ???
#1  0x7fd8d382d503 in ???
#2  0x7fd8d26ecf1f in ???
#3  0x7fd8d283cf2d in ???
#4  0x5601f20976c8 in ???
#5  0x5601f205eb9e in ???
#6  0x5601f0b109dd in ???
#7  0x5601f0b034fc in ???
#8  0x5601f0bddfa0 in ?

In [18]:
parse_nwchem_output("data/caffeine.nw", "data/caffeine.output")

Number of Atoms:  24
Number of Orbitals:  80
SCF Energy:  -627.628748901350


IndexError: list index out of range

In [35]:
cat data/caffeine.nw


start caffeine

echo
memory stack 1000 mb heap 100 mb global 1000 mb noverify

geometry units au
symmetry c1
H -3.3810087277706735 -1.5457442099995116 0.2984523788341226
N -0.7964845233221604 1.4639959814937358 -0.1945708240384304
C 0.7432359364766733 -0.331715224806486 0.1522725193947619
N -1.5143911338824416 -0.8197651187313808 -0.2679161053613439
C -0.2435856781498955 -1.3415705040281563 -0.008840405100768845
C -1.8364037412156626 0.5501718165354377 -0.3686098579190236
C 0.4706285001157109 0.9994673823861003 0.06217304835112866
N 2.0819601662060925 -0.4004580782763669 0.41157687484153943
O -2.993095343715203 0.9142024869731757 -0.601279925518949
C -2.610635395601299 -1.750414455436597 -0.4523080122207022
O 0.012686209241972442 -2.539039119021219 0.0743319647309779
H 3.5848035713403372 1.130219268834055 0.6571401809668731
C -1.0437187983873932 2.8915233868735215 -0.28251923669790563
H -3.045844010065921 -1.5951082346536627 -1.4450211445676504
H -2.2969418031620448 -2.793301939339378

In [44]:
parse_nwchem_output("data/caffeine.nw", "data/caffeine.output")

IndexError: list index out of range